In [18]:
!pip install -r requirements.txt

In [1]:
import pandas as pd

# ※ Dataset 로드

#1. path 설정
train_path = '../평가용 데이터셋/1.Training/라벨링데이터/'
val_path = '../평가용 데이터셋/2.Validation/라벨링데이터/'
test_path = '../평가용 데이터셋/3.Test/라벨링데이터/'

#2-1. Dataset(전향)
train_for = pd.read_csv(train_path + '호흡기전향_학습.csv',index_col=None)
val_for = pd.read_csv(val_path + '호흡기전향_평가.csv',index_col=None)
test_for = pd.read_csv(test_path + '호흡기전향_테스트.csv',index_col=None)

#2-2. Dataset(후향)
train_bck = pd.read_csv(train_path + '호흡기후향_학습.csv',index_col=None)
val_bck = pd.read_csv(val_path + '호흡기후향_평가.csv',index_col=None)
test_bck = pd.read_csv(test_path + '호흡기후향_테스트.csv',index_col=None)

#3. 제외컬럼 설정
drop_list = ['train', 'val', 'test']
drop_list_for = [train_for, val_for, test_for]
drop_list_bck = [train_bck, val_bck, test_bck]
for name, i, j in zip(drop_list, drop_list_for, drop_list_bck):
    i.drop_duplicates()
    globals()["{}".format(i)] = i.drop(
        columns=["PI_PatientId",
        "DI_Date",
        "DI_Name",
        "DI_Provider",
        'PEGS_Rt',
        'PEGS_Lt',
        'QE5_TotalScore',
        'QC_TotalScore',
        'QSF_TotalScore'], axis=1, inplace=True)
    j.drop_duplicates()
    globals()["{}".format(j)] = j.drop(
        columns=["PI_PatientId",
        "DI_Date",
        "DI_Name",
        "DI_Provider",
        'PEGS_Rt',
        'PEGS_Lt',
        'QE5_TotalScore',
        'QC_TotalScore',
        'QSF_TotalScore'], axis=1, inplace=True)
    globals()["{}".format(name)] = pd.concat([i,j], join='inner')

In [2]:
#4. Dataset 사례수 확인
print('train dataset:', len(train), '\n' 'validation dataset:', len(val), '\n' 'test dataset:', len(test))

train dataset: 225 
validation dataset: 28 
test dataset: 29


In [3]:
#5. Category 변수 인코딩(양호/허약 → 0/1로 변환)
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

train_label = train['Label'].copy()
train_df_label = LE.fit_transform(train_label)
train_df_y = train["PEGS_BestScore"].copy()
train_df_x = train.copy()
train_df_x = train_df_x.drop(columns = ["PEGS_BestScore", "Label"])

val_label = val['Label'].copy()
val_df_label = LE.fit_transform(val_label)
val_df_y = val["PEGS_BestScore"].copy()
val_df_x = val.copy()
val_df_x = val_df_x.drop(columns = ["PEGS_BestScore", "Label"])

test_label = test['Label'].copy()
test_df_label = LE.fit_transform(test_label)
test_df_y = test["PEGS_BestScore"].copy()
test_df_x = test.copy()
test_df_x = test_df_x.drop(columns = ["PEGS_BestScore", "Label"])

In [4]:
from catboost import CatBoostRegressor
import numpy as np

#6. 모델링(Regression)
cbr = CatBoostRegressor(
    iterations=10000,
    learning_rate=1,
    depth=15,
    # l2_leaf_reg=20,
    # bootstrap_type='Bernoulli',
    subsample=0.6,
    eval_metric='MAE',
    metric_period=1,
    od_type='Iter',
    od_wait=45,
    random_seed=17,
    allow_writing_files=True
)

#7. 모델 Train 수행
cbr.fit(
    train_df_x, np.log1p(train_df_y),
    eval_set=[(train_df_x, np.log1p(train_df_y)),(val_df_x, np.log1p(val_df_y))],
    use_best_model=True,
    verbose=True)

pred = cbr.predict(test_df_x)

0:	learn: 0.1786944	test: 0.1786944	test1: 0.1886128	best: 0.1886128 (0)	total: 223ms	remaining: 37m 6s
1:	learn: 0.1311417	test: 0.1311417	test1: 0.1705804	best: 0.1705804 (1)	total: 262ms	remaining: 21m 50s
2:	learn: 0.1188443	test: 0.1188443	test1: 0.1672095	best: 0.1672095 (2)	total: 336ms	remaining: 18m 39s
3:	learn: 0.1032970	test: 0.1032970	test1: 0.1732517	best: 0.1672095 (2)	total: 722ms	remaining: 30m 4s
4:	learn: 0.0903859	test: 0.0903859	test1: 0.1812889	best: 0.1672095 (2)	total: 1.33s	remaining: 44m 11s
5:	learn: 0.0814836	test: 0.0814836	test1: 0.1782588	best: 0.1672095 (2)	total: 1.89s	remaining: 52m 22s
6:	learn: 0.0619859	test: 0.0619859	test1: 0.1801808	best: 0.1672095 (2)	total: 2.45s	remaining: 58m 22s
7:	learn: 0.0521604	test: 0.0521604	test1: 0.1790050	best: 0.1672095 (2)	total: 3.03s	remaining: 1h 3m 9s
8:	learn: 0.0441888	test: 0.0441888	test1: 0.1788046	best: 0.1672095 (2)	total: 3.56s	remaining: 1h 5m 48s
9:	learn: 0.0408644	test: 0.0408644	test1: 0.1776556	b

In [5]:
#8. 예측변수 후처리 : Regression을 통해 나온 예측값(수치변인)을 아래 기준에 맞게 Category(범주변인)로 재정의
test_df_x["PEGS_pred"] = np.expm1(pred)
test_df_x.loc[(test_df_x.PI_Sex==1)&(test_df_x.PEGS_pred<28),"new_Label"] = "허약"
test_df_x.loc[(test_df_x.PI_Sex==1)&(test_df_x.PEGS_pred>=28),"new_Label"] = "양호"
test_df_x.loc[(test_df_x.PI_Sex==2)&(test_df_x.PEGS_pred<18),"new_Label"] = "허약"
test_df_x.loc[(test_df_x.PI_Sex==2)&(test_df_x.PEGS_pred>=18),"new_Label"] = "양호"

In [6]:
test_df_x['PEGS_pred']

0     30.480678
1     24.007398
2     25.783102
3     32.262189
0     26.631844
1     30.893120
2     29.908206
3     34.346314
4     27.688946
5     20.425055
6     29.291001
7     27.680218
8     28.291635
9     27.948725
10    26.698299
11    34.346314
12    40.730789
13    24.040026
14    27.845540
15    28.411169
16    19.259626
17    30.651854
18    37.485535
19    32.584055
20    20.637507
21    26.698299
22    35.882216
23    30.651854
24    23.262774
Name: PEGS_pred, dtype: float64

In [7]:
#9. 새로운 Category 변수 인코딩(양호/허약 → 0/1로 변환)
LE = LabelEncoder()
new_df_label = LE.fit_transform(test_df_x["new_Label"])
new_df_label

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0])

In [8]:
#10. 모델 평가
from sklearn.metrics import classification_report
test_labellist = pd.DataFrame(test_df_label)
cls = classification_report(test_labellist, new_df_label)
# 0 : 양호, 1 : 허약
print(cls)

              precision    recall  f1-score   support

           0       0.83      0.95      0.89        21
           1       0.80      0.50      0.62         8

    accuracy                           0.83        29
   macro avg       0.82      0.73      0.75        29
weighted avg       0.82      0.83      0.81        29



In [9]:
#11. 평가 결과물 저장
def printsave(*a):
    file = open('./eval_f1_score.txt','a')
    print(*a)
    print(*a,file=file)
    file.close()

printsave(cls)

              precision    recall  f1-score   support

           0       0.83      0.95      0.89        21
           1       0.80      0.50      0.62         8

    accuracy                           0.83        29
   macro avg       0.82      0.73      0.75        29
weighted avg       0.82      0.83      0.81        29



In [10]:
#12. 모델 저장
cbr.save_model("./model_호흡기.cbm",
               format = "cbm",
               export_parameters=None,
               pool = None)

In [11]:
#13. 모델 불러오기
cbr_load = CatBoostRegressor()
cbr_load.load_model("./model_호흡기.cbm")

In [12]:
#14. 새로운 데이터를 로드하여 모델 적용
new_path = "../평가용 데이터셋/3.Test/라벨링데이터/" # "파일 경로 입력"
new_load_for = pd.read_csv(new_path + '호흡기전향_테스트.csv',index_col=None)
new_load_bck = pd.read_csv(new_path + '호흡기후향_테스트.csv',index_col=None)

# train = pd.concat([train_for, train_bck], join='inner')

new_drop_list = [new_load_for, new_load_bck]

for i in new_drop_list:
    i.drop_duplicates()
    globals()["{}".format(i)] = i.drop(
        columns=["PI_PatientId",
        "DI_Date",
        "DI_Name",
        "DI_Provider",
        'QE5_TotalScore',
        'QC_TotalScore',
        'QSF_TotalScore'], axis=1, inplace=True)

new_load = pd.concat([new_load_for, new_load_bck], join='inner')
new_load = new_load.drop(columns = ["PEGS_BestScore", "Label"])

pred = cbr_load.predict(new_load)

new_load["PEGS_pred"] = np.expm1(pred)
new_load.loc[(new_load.PI_Sex==1)&(new_load.PEGS_pred<28),"new_Label"] = "허약"
new_load.loc[(new_load.PI_Sex==1)&(new_load.PEGS_pred>=28),"new_Label"] = "양호"
new_load.loc[(new_load.PI_Sex==2)&(new_load.PEGS_pred<18),"new_Label"] = "허약"
new_load.loc[(new_load.PI_Sex==2)&(new_load.PEGS_pred>=18),"new_Label"] = "양호"

C:\Users\sweetk\AppData\Local\Temp\ipykernel_7724\383812656.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_load["PEGS_pred"] = np.expm1(pred)
C:\Users\sweetk\AppData\Local\Temp\ipykernel_7724\383812656.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_load.loc[(new_load.PI_Sex==1)&(new_load.PEGS_pred<28),"new_Label"] = "허약"


In [13]:
#15. 새로운 데이터의 결과값 예측(가장 마지막 컬럼인 new_label)
new_load

,PI_Age,PI_Sex,MI_ChronicObstructiveLungDisease,MI_Asthma,MI_Bronchiectasis,MI_InterstitialLung,MI_LungCancer,MI_TuberculousLung,MI_RestrictedLungDiseaseDueToMusculoskeletalDisorders,MI_PulmonaryArterialHypertension,...,QIB_NoOfTimesPerWeek,QIB_LevelOfExercise,QBE_Time,QBE_NoOfTimesPerDay,QBE_NoOfTimesPerWeek,QO_Time,QO_NoOfTimesPerDay,QO_NoOfTimesPerWeek,PEGS_pred,new_Label
0,78,1,1,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,4.0,1.0,5.0,30.480678,양호
1,65,2,0,0,0,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.007398,양호
2,61,2,1,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.783102,양호
3,70,1,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.262189,양호
0,62,2,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.631844,양호
1,64,1,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.893120,양호
2,70,1,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.908206,양호
3,58,1,0,0,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.346314,양호
4,48,1,1,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.688946,허약
5,71,2,0,1,0,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.425055,양호
